In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [16]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
#URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_Europe'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']

In [46]:
# init dicts
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages[:]:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Skip useless/empty stuff
        if Tags.span == None:
            continue

        # Set chapter variable    
        Chapter = Tags.span.attrs['id']

        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':


            # Get species name
            Species = soup.title\
                            .string.split(' - ')[0]\
                            .rstrip(' ')

            # Create list for key
            DescriptionData[Species] = []
            NonDescriptionData[Species] = []

            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    DescriptionData[Species] += SentenceList

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    NonDescriptionData[Species] += SentenceList

In [27]:
'''
URL = 'https://en.wikipedia.org/wiki/Brown_bear'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for Tags in soup.find_all('h2'):

    # Skip useless/empty stuff
    if Tags.span == None:
        continue

    # Set chapter variable    
    Chapter = Tags.span.attrs['id']

    # Check if the chapter is description (or similar)
    if Chapter == 'Characteristics'or \
       Chapter == 'Description' or \
       Chapter == 'Appearance':


        # Get species name
        Species = soup.title\
                        .string.split(' - ')[0]\
                        .rstrip(' ')

        # Create list for key
        DescriptionData[Species] = []
        NonDescriptionData[Species] = []

        # Get the next sibling (text)
        for Text in Tags.find_next_siblings('p'):

            # Add description data to dict
            if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                DescriptionData[Species] += SentenceList
                
            # Add non description data to dict
            elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                NonDescriptionData[Species] += SentenceList
'''

In [50]:
len(DescriptionData['Brown bear'])

52

In [53]:
df_True = pd.DataFrame.from_dict(DescriptionData, orient='index')
df_False = pd.DataFrame.from_dict(NonDescriptionData, orient='index')

In [60]:
df_True.loc['Brown bear', 0:52]

0     The brown bear is the most variable in size of...
1     The typical size depends upon which population...
2     This is in part due to sexual dimorphism, as m...
3     Individual bears also vary in size seasonally,...
4     Therefore, a bear may need to be weighed in bo...
5     The normal range of physical dimensions for a ...
6     The tail is relatively short, as in all bears,...
7     The smallest brown bears, females during sprin...
8     Interior brown bears are generally smaller tha...
9     The average weight of adult male bears from 19...
10                Brown bears are often not fully brown
11    They have long, thick fur, with a moderately l...
12    In India, brown bears can be reddish with silv...
13    Even within well-defined subspecies, individua...
14    North American grizzlies can be dark brown (al...
15    The common name "grizzly" stems from their typ...
16    Apart from the cinnamon subspecies of the Amer...
17    americanus cinnamonum), the brown bear is 

In [55]:
df_True.to_pickle("df_true.pkl")
df_False.to_pickle("df_false.pkl")